## `KGBuilder` 知識抽取範例
示範使用 `KGBuilder` 對文章進行資訊抽取
1. 文章前處理
2. 事件三元組抽取
3. 實體抽取

---

#### (1) 文章前處理
[Input] 
- format: `.csv`
- 可指定想要跑前處理的 `.csv` 檔 
  - headers: `title`, `url`, `meta_keywords`, `crawling_time`, `article`, `publish_time`, `media`

[Output]
- format: `.csv`
- 前處理過後的新聞會以 `.csv` 儲存至 `./article_data/` 底下
- 檔名會加入 `cleansed-` 前綴、當天日期後綴

![](https://i.imgur.com/O58RJSn.png)

In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import warnings
warnings.filterwarnings("ignore")

from KGBuilder.config import *
from KGBuilder import data_utils

In [7]:
# 載入文章資料集
article_filepath = "AllNews-20220610.csv"
article_data = data_utils.load_article(article_filepath=article_filepath)

# 進行文章前處理
preprocessed_data = data_utils.do_preprocess(data=article_data)
print(preprocessed_data.shape)
print(preprocessed_data.head())


====== Load Data ======
title            0
url              0
meta_keywords    4
crawling_time    0
article          0
publish_time     0
media            0
dtype: int64
Shape of article data: (161, 7)
====== Start Preprocessing ======
Article data cleansed. 
                                     title  \
0   南帝第一季合併獲利5.78億元，每股稅後1.17元 - MoneyDJ理財網   
1        台端董事會決議解除經理人競業禁止之限制案 - MoneyDJ理財網   
2   飛寶企業第一季獲利1608萬元，每股稅後0.51元 - MoneyDJ理財網   
3     東森購物網前4月EPS 5.96元，寫同期新高 - MoneyDJ理財網   
4  長榮鋼第一季合併獲利3.43億元，每股稅後0.82元 - MoneyDJ理財網   

                                                 url  \
0  https://www.moneydj.com/kmdj/news/newsviewer.a...   
1  https://www.moneydj.com/kmdj/news/newsviewer.a...   
2  https://www.moneydj.com/kmdj/news/newsviewer.a...   
3  https://www.moneydj.com/kmdj/news/newsviewer.a...   
4  https://www.moneydj.com/kmdj/news/newsviewer.a...   

                    meta_keywords            crawling_time  \
0                              南帝  2022-05-09 06:23:47.643   
1 

#### (2) 事件三元組抽取
示範對一篇文章進行事件抽取

[Input] 
- format: `str`
- 想要跑事件三元組抽取的文章

[Output]
- format: `dict`
- 事件三元組的抽取結果

In [2]:
# 對文本進行基於句法分依存分析的事件三元組標註
from KGBuilder.EventTriplet.ltp import TripleExtractor, CausalityExractor
triplet_extractor = TripleExtractor(model_tag="base2")
causality_extractor = CausalityExractor(model_tag="base2")


In [15]:
article = preprocessed_data["article"][100]
article


'財政部今天公布 4 月出口金額為 414.6 億美元，年增 18.8%，連 22 紅；官員指出，儘管世界經濟遭遇逆風，台灣 4 月出口額仍在供應鏈瓶頸改善和遞延出貨效應等三大利多挹注下，創歷年單月第三高紀錄。展望後市，財政部表示，5月出口規模仍將超過400億美元，預估會落在419億至434億美元，年增率區間為12%~16%。（Source：財政部）財政部今天公布4月海關進出口貿易初步統計，4月出口金額為414.6億美元、年增18.8%，連22紅達陣，續寫史上正成長次長週期；進口金額為365.5億美元，為歷年單月次高，年增26.7%；累計1~4月出口1624億美元，較去年同期增22.3%。財政部統計處處長蔡美娜分析出口概況，今年2月底以來，全球地緣政治風險提高、中國疫情升溫，干擾世界經濟，但所幸台灣4月出口依然在科技創新與數位轉型潮流、供應鏈瓶頸改善和遞延出貨效應挹注，加上國際原物料行情上揚推升出口產品價格，三大有利因素加持下，展現相當活力。觀察出口前11個主要貨品項目，光學器材、塑橡膠及製品呈現年減格局，其餘9類貨品呈現成長；蔡美娜指出，光學器材出口約9.2億美元、年減15.6%，呈現連續6個月下降，出口規模跌到22個月低點，主因是面板供需結構偏弱、客戶端採購保守，造成價格跌幅擴大。電子零組件4月出口額達172.1億美元、年增27.5%，創單月第二高紀錄，連36紅，為史上第二長正成長週期；蔡美娜說明，主要是受到高效能運算、車用電子需求依然強勁，加上晶片漲價效益挹注。資通與視聽產品方面，4月出口51.88億美元、年增10.2%，但各地區表現有所落差，對美國出口依舊熱絡，但中國與香港、東協買氣不如以往；蔡美娜表示，可能與家庭端消費減弱、疫情紅利消退，中國經濟放緩和封控有關。至於傳統類貨品出口，以礦產品年增率達59.4%為最高，蔡美娜指出，主要是礦產品報價隨國際原油行情而水漲船高；運輸工具出口年增26.5%，在全球新車產能不足、近年健身概念提升下，間接帶動歐美地區汽車、腳踏車維修市場熱絡，擴大對相關零附件下單。基本金屬及製品4月出口33.44億美元，蔡美娜說，歐美買氣維持高檔，反映原物料成本上揚，出口年增26.1%；化學品受惠於價量齊揚，出口金額23.11億美元，刷新單月紀錄。展望後市，蔡美娜指出，5月工作天數增加，順季節性態樣推估，5月出口水準可站穩400億美元之上，

In [16]:
# 進行事件三元組抽取
triplets = triplet_extractor.triples_main(article)
triplets

[['財政部', '公布', '4 月出口金額為414.6 億美元年增18.8%'],
 ['4 月出口金額', '為', '414.6 億美元'],
 ['4 月出口金額', '增', '18.8%'],
 ['官員', '指出', '儘管世界經濟遭遇逆風台灣4 月出口額仍在供應鏈瓶頸改善和遞延出貨效應三大利多挹注下創歷年單月第三高紀錄'],
 ['世界經濟', '遭遇', '逆風'],
 ['台灣4 月出口額', '創', '歷年單月第三高紀錄'],
 ['財政部', '表示', '5月出口規模仍將超過400億美元預估會落在419億至434億美元年增率區間為12%16%'],
 ['出口規模', '超過', '400億美元'],
 ['年增率區間', '為', '12%16%'],
 ['財政部', '公布', '4月海關進出口貿易初步統計'],
 ['4月出口金額', '為', '414.6億美元'],
 ['4月出口金額', '增', '18.8%'],
 ['進口金額', '為', '365.5億美元'],
 ['進口金額', '為', '歷年單月次高'],
 ['財政部統計處處長蔡美娜', '分析', '出口概況'],
 ['光學器材塑橡膠及製品', '呈現', '年減格局'],
 ['其餘9類貨品', '呈現', '成長'],
 ['蔡美娜',
  '指出',
  '光學器材出口約9.2億美元年減15.6%呈現連續6個月下降出口規模跌到22個月低點主因是面板供需結構偏弱客戶端採購保守造成價格跌幅擴大'],
 ['光學器材出口', '呈現', '連續6個月下降'],
 ['出口規模', '跌', '22個月低點'],
 ['主因', '是', '面板供需結構偏弱客戶端採購保守造成價格跌幅擴大'],
 ['電子零組件4月出口額', '達', '172.1億美元'],
 ['電子零組件4月出口額', '創', '單月第二高紀錄'],
 ['電子零組件4月出口額', '為', '史上第二長正成長週期'],
 ['蔡美娜', '說明', '主要是受到高效能運算車用電子需求依然強勁加上晶片漲價效益挹注'],
 ['各地區表現', '有所', '落差'],
 ['蔡美娜', '表示', '可能與家庭端消費減弱疫情紅利消退中國經濟放緩和封控有關'],
 ['礦產品年增

In [18]:
_causalities = causality_extractor.extract_main(article)
causalities = []
for _cau in _causalities:
    cause = ''.join([word.split('/')[0] for word in _cau['cause'].split(' ') if word.split('/')[0]])
    tag = _cau["tag"]
    effect = ''.join([word.split('/')[0] for word in _cau['effect'].split(' ') if word.split('/')[0]])
    causalities.append([cause, tag, effect])

causalities

[['6%，呈現連續6個月下降，出口規模跌到22個月低點，主因是面板供需結構偏弱、客戶端採購保守，', '造成', '價格跌幅擴大。']]

#### (3) 實體抽取
1. 先從 GCS 下載 DST 訓練的 `FiNER` 模型及 token class schema
2. 就文本進行 `CKIPWrapper` NER 實體標註
3. 就文本進行 `FiNER` NER 實體標註

[Input]
- format: `str`
- 要進行實體抽取的文本

[Output]
- format: `dict`
- 實體抽取的分類結果


In [19]:
# 從 GCS 下載模型至本地端(原始的 bert-base-chinese 以及 已訓練好的 FiNER & FinRE)
from KGBuilder.data_utils import gcs_downloader
for model_dir in ["baseBert", "FiNER"]:
    download_to_dir = gcs_args[f"local_model_dir_{model_dir}"]
    source_dir = os.path.join(gcs_args["gcs_model_directory"], download_to_dir)
    gcs_downloader(source_dir, download_to_dir)

# 將 FiNER 所需的 slot_label.txt 從 dictionary 資料夾移過來
filename = "slot_label.txt"
if not os.path.exists(os.path.join(args["model_dir"], "input", filename)):
    shutil.copyfile(
        os.path.join(gcs_args["gcs_dict_directory"], filename), 
        os.path.join(args["model_dir"], "input", filename)
    )


5it [00:00, 123.91it/s]


bert-base-chinese/bert_model.ckpt.data-00000-of-00001 exists, please check before download again
bert-base-chinese/bert_model.ckpt.index exists, please check before download again
bert-base-chinese/bert_model.ckpt.meta exists, please check before download again
bert-base-chinese/config.json exists, please check before download again
bert-base-chinese/vocab.txt exists, please check before download again
Files download to bert-base-chinese successfully.


3it [00:00, 96.76it/s]

FiNER/config.json exists, please check before download again
FiNER/pytorch_model.bin exists, please check before download again
FiNER/training_config.bin exists, please check before download again
Files download to FiNER successfully.


In [23]:
# 初始化 CKIP 模型服務
"""
    CKIPWrapper 是 DST Abao 將 CKIPTagger 及 CKIP Transformers 打包合併的服務
    目前接口提供 NER 模型的呼叫
    （將 CKIPTagger 及 CKIP Transformers 兩者的 NER 結果聯集後返回）
"""

from KGBuilder.ckipNER.ckip import CKIPWrapper
ckip = CKIPWrapper()

2022-06-13 00:08:39.665381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 00:08:39.666074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 00:08:39.667039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 00:08:39.667570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 00:08:39.668508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

{'PERCENT': ['18.8%'], 'ORG': ['財政部']}

In [25]:
# 執行 CKIPWrapper NER 實體抽取
input_text = triplets[0]
_res_ckip = ckip.ner_predict(input_text=[article])[0]["entity_text"]
_res_ckip

100%|██████████| 1/1 [00:00<00:00, 3919.91it/s]


{'CARDINAL': ['2', '9', '36', '11', '22', '三'],
 'MONEY': ['1624億美元',
  '9.2億美元',
  '172.1億美元',
  '23.11億美元',
  '419億至434億美元',
  '33.44億美元',
  '414.6 億美元',
  '414.6億美元',
  '365.5億美元',
  '400億美元',
  '51.88億美元',
  '22紅'],
 'ORG': ['財政部統計處', 'Source', '財政部', '東協'],
 'DATE': ['5月',
  '4月',
  '今天',
  '4 月',
  '單月',
  '6個月',
  '去年',
  '22個月',
  '1~4月',
  '2月底',
  '今年2月底'],
 'ORDINAL': ['第二', '第三'],
 'GPE': ['台灣', '美', '美國', '中國', '香港', '歐'],
 'PERCENT': ['12%',
  '26.5%',
  '18.8%',
  '10.2%',
  '26.1%',
  '15.6%',
  '12%~16%',
  '27.5%',
  '26.7%',
  '59.4%',
  '22.3%',
  '16%'],
 'PERSON': ['蔡美娜', '張璦'],
 'LOC': ['歐美', '歐'],
 'NORP': ['歐']}

In [27]:
# 初始化 FiNER 模型
from KGBuilder.FiNER.args import Args
args_finer = Args()
from KGBuilder.FiNER.trainer import FiNER
from KGBuilder.FiNER.utils import (
    init_logger,
    load_tokenizer,
    set_seed,
)
args_finer.do_pred = True  # 指定模式為：預測
args_finer.use_crf = True  # 指定使用 CRF layer
args_finer.task = "input"  # token class schema file 位在的資料夾名稱
args_finer.model_dir = "FiNER"

set_seed(args_finer)  # 隨機亂數初始化
tokenizer = load_tokenizer(args_finer)  # 載入 bert tokenizer (original BERT)
finer = FiNER(args_finer)  # 參數初始化
finer.load_model()  # 載入 FiNER 模型


In [29]:
# 先將文章 tokenize
text_tokens = " ".join([word for word in article])
_res_finer = finer.predict([text_tokens], tokenizer)[0]["entity_text"]
# 整理名稱
_res_finer["EVENT"] = _res_finer.pop("EVE")
_res_finer["PRODUCT"] = _res_finer.pop("GOO")

_res_finer


torch.Size([1, 200])


Predicting: 100%|██████████| 1/1 [00:00<00:00, 17.77it/s]


finished prediction!!!


1it [00:00, 1141.31it/s]
100%|██████████| 1/1 [00:00<00:00, 11748.75it/s]

json file has stored.


{'ORG': ['財政部'],
 'ART': [],
 'OTH': ['出口金額',
  '世界經濟',
  '逆風',
  '出口額',
  '供應鏈瓶頸',
  '遞延',
  '效應',
  '紀錄',
  '後市',
  '出口規模',
  '年增率',
  'Source',
  '貿易'],
 'EVENT': [],
 'PRODUCT': []}